# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

# keep last points for smothing when drawing lanes
# specially useful when working on videos:

prev_iter = {
    'll' : None,
    'rl' : None
}

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def lane_endpoints(lane_info, y_max, y_min, lane_prefix):
    """
    Computing end points for a lane given information
    about all points that conform with the slope criteria.
    """
    
    global prev_iter
    li = lane_info
    avg_slope = np.mean(li['slopes'])
    x_avg, y_avg = (int(np.mean(li['xs'])), int(np.mean(li['ys'])))
    x_max = int((y_max - y_avg)/avg_slope + x_avg)
    x_min = int((y_min - y_avg)/avg_slope + x_avg)

    #smothing of line
    if prev_iter[lane_prefix]:
        pp = prev_iter[lane_prefix]
        (x_min, y_min, x_max, y_max) = (int((x_min + pp[0])/2),
                                        int((y_min + pp[1])/2),
                                        int((x_max + pp[2])/2),
                                        int((y_max + pp[3])/2))
    prev_iter[lane_prefix] = (x_min, y_min, x_max, y_max)
    return (x_min, y_min, x_max, y_max)

    
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    ll = {'slopes': [], 'xs': [], 'ys': []}
    rl = {'slopes': [], 'xs': [], 'ys': []}
    y_max = image.shape[0]
    
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            #cv2.line(img, (x1, y1), (x2, y2), color, 4)
            m = (y2-y1)/(x2-x1)
            if (m < -0.2):
                ll['slopes'].append(m)
                ll['xs'].extend([x2, x1])
                ll['ys'].extend([y2, y1])
            elif (m > 0.2): 
                rl['slopes'].append(m)            
                rl['xs'].extend([x2, x1])
                rl['ys'].extend([y2, y1])

    ll_y_min = np.amin(ll['ys']) if ll['ys'] else y_max
    rl_y_min = np.amin(rl['ys']) if rl['ys'] else y_max
    
    y_min = min(ll_y_min, rl_y_min)

    # handling left lane
    if ll['xs']:
        (x1, y1, x2, y2) = lane_endpoints(ll, y_max, y_min, 'll')
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    # handling right lane
    if rl['xs']:
        (x1, y1, x2, y2) = lane_endpoints(rl, y_max, y_min, 'rl')
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)

    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# Putting together the pipeline:

def find_lanes_in_image(image):
    """
    `image` raw image on which we will draw lanes.
    
    returns image with left and right lanes drawn on.
    """
    
    imshape = image.shape
    gray = grayscale(image)
    blured = gaussian_blur(gray, 5)
    edges = canny(blured, 50, 200)

    poly_vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320),
                          (imshape[1],imshape[0])]],
                        dtype=np.int32)

    masked_edges = region_of_interest(edges, poly_vertices)
    
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 30 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    
    hlines = hough_lines(masked_edges, rho , theta,
                        threshold, min_line_len, max_line_gap) 

    return weighted_img(hlines, image, α=0.8, β=1., λ=0.)

image_with_lanes = find_lanes_in_image(image)

plt.figure(),plt.imshow(image)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.figure(), plt.imshow(image_with_lanes, cmap='gray')
plt.title('Lines Image'), plt.xticks([]), plt.yticks([])

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os

test_dir = "test_images/"
for image_file in os.listdir(test_dir):
    global prev_iter
    if ( not image_file.endswith('.jpg') or image_file.startswith('Lines')):
        continue

    # reset smoothing
    prev_iter = {
        'll' : None,
        'rl' : None
    }
    
    new_image_file = "Lines" + image_file
    new_image_file_path = test_dir + new_image_file
    os.remove(new_image_file_path) if os.path.exists(new_image_file_path) else None
    
    #display results
    image = mpimg.imread(test_dir + image_file)
    image_with_lanes = find_lanes_in_image(image)
    mpimg.imsave(new_image_file_path, image_with_lanes, format='jpg')
    
    plt.figure()
    plt.subplot(121),plt.imshow(image)
    plt.title(image_file), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(image_with_lanes)
    plt.title(new_image_file), plt.xticks([]), plt.yticks([])

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    return find_lanes_in_image(image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


# MY REFLECTIONS:

This first project was lots of fun.. so I look forward to the next ones.

I describe shortly how my solution is implemented:

The whole logic is encapsulated into one single method: `find_lanes_in_image`. This method takes as input an array representation of an image and returns the same image with left and right lanes drawn on it.

The pipeline implementad operates as follows:

#### Step 1: 
First the color scale of the image is transformed to gray scale which will ease the detection of lines of any color in a future step. After that noise and details are reduced by using Gaussian smoothing. At this point the image going through the pipelines looks like this:

---
<figure>
 <img src="./Reflections/blured.jpg" width="380" alt="GrayBlurred" />
</figure>

#### Step 2:
Then, Canny Edge detection is applied to the image in gray scale to detect all edges. The idea behind this is that the gradient is used to see how fast pixels change in different areas of the image and in which direction they change most rapidly. On top of that, we would like to filter the areas on which we will work on for the rest of the pipeline because we know in which area of the image our lanes will show up. So we mask all we've done so far. The result looks then like:

---
<figure>
 <img src="./Reflections/canny_masked.jpg" width="380" alt="Canny" />
</figure>

#### Step 3:
Now, a very nice trick is applied for detecting straight lines out of edges: Hough transformation. The Hough transformation takes all edge points to a different space to try to find the parameters $m$ and $b$ that fine lines of the form: $y = m*x + b$. As a result we obtain end points of lines that we can use to infer left and right lanes. At this stage of the pipeline, the processed image looks like this:

---
<figure>
 <img src="./Reflections/hough.jpg" width="380" alt="Canny" />
</figure>

#### Step 4:
You see that we have a bunch of lines. Somehow we have to make sense out of them and try to figure out where exactly our left and right lanes are. For this, all line segment are processed using the following huristic:

* First, each line segment either has posstive or negative slope. If the slope is negative, then the segment is probably part of the left lane. Otherwise, part of the right lane. During this categorization all those lines with slope within the range $[-0.2, 0.2]$ are excluded. If you think about it, it is highly unlikely that a lane with slope 0.1 (almost 0, in other words, a horizontal line) would be a lane our car should follow.
* Second, for each left and right lane, average quantities are computed: slope and mean-point.
* Given the avg. slope and avg. point, we can find exactly the two endpoints that we need to draw the each lane. We know the min and max y for each of the desired endpoints (y max being the edge of the image itself and y min being the minimum y that contributed to the average point) so we can use the formulas $x_{min} = (y_{min} - y_{avg})/m_{avg} + x_{avg}$ and $x_{max} = (y_{max} - y_{avg})/m_{avg} + x_{avg}$.
* For the case on which we are processing an stream of images (a video for example) we take endpoints for each lane and average them with the endpoints we saw from the previous image frame. The purpose of this is to smooth drastic changes from frame to frame.

The result is one straight line for each lane:

---
<figure>
 <img src="./Reflections/lanes.jpg" width="380" alt="Lanes" />
</figure>

#### Step 5:
Finally we need to merge original image with our resulting lanes.

---
<figure>
 <img src="./Reflections/merged.jpg" width="380" alt="Result" />
</figure>



### Current implementation limitations:

Clearly this implementation fails on the video challenge and these are my thoughts on it.

* The current strategy of converting the original image to a gray scale might not be the best one when there are for example big shadows or other kind of noise that affect brightness directly. Probably another color space would be more tolerant to this situations. I see people mentioning on the forums that a good alternative would be to use HSL or HSV.

* Within the pipeline I use a fixed ROI ( Region of Interest ), which might not be the best for when the car starts to approach curves or for when there are median strips within the ROI. As well, it might be that in order to have the camara capture proximate portions of lanes it would capture as well a good portion of the car itself needing a adjustment of the ROI. A good thing to try here would be to dinamically select a good ROI within the image.

* I am not sure how the algorithm would react if there is another car within the ROI. Probably the number of lines detected would increase dramatically ( given the rectangular shapes of cars ). In this case a better heuristic to discriminate lines that dont form part of a lane would be needed. A starting point would be to ignore line segments that are too steep ( aproximately vertical lines ).

* The smoothing mechanism can defenitely be improved. At the moment I am just keeping the coordinates of previous endpoints the algorithm found, which is rather simplistic. Not sure what the best approach would be but probably extending the amount of points kept as history plus a regularized update for current positions might be a good start.

Some scenarios on which I am completly sure my algorithm would miserably fail:

* When driving on roads without marked lanes. In my country, when you find yourself in the country side usually you drive on roads that have no lanes. I think that in this case a broader ROI is needed.

* When you find yourself on a traffic jam the car in front of you blocks your camera from **seing** the lanes. Probably the car in this case **pays attention to other sensors(?)**.

* When driving backwards :).


## Submission

If you're satisfied with your video outputs it's time to submit!


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))